# Google Play Store Data Analysis - Training

> ## 📦 1. Importing Libraries

In [691]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import webbrowser
import os

In [ ]:
nltk.download('vader_lexicon')

> ## 📁 2. Data Loading

In [ ]:
app_df = pd.read_csv('.\Play Store Data.csv')
app_df.head()

In [ ]:
reviews_df = pd.read_csv('.\\User Reviews.csv')
reviews_df.head()

> * Missing values

In [ ]:
app_df.isnull()

In [ ]:
reviews_df.isnull()

> ## 🧹 3. Data Cleaning & Preprocessing

In [697]:
app_df = app_df.dropna(subset = ['Rating'])

In [ ]:
for column in app_df.columns:
    app_df[column].fillna(app_df[column].mode()[0], inplace = True)

app_df.drop_duplicates(inplace = True)

app_df = app_df[app_df['Rating']<=5]

In [699]:
reviews_df.dropna(subset=['Translated_Review'],inplace=True)

In [ ]:
app_df.dtypes

* Removing + and , from the Install column of app_df

In [608]:
#Convert the Installs columns to numeric by removing commas and +
app_df['Installs']=app_df['Installs'].str.replace(',','').str.replace('+','').astype(int)

In [702]:
#Convert Price column to numeric after removing $
app_df['Price']=app_df['Price'].str.replace('$','').astype(float)

In [ ]:
app_df.dtypes

* Merging both app and review table by inner join on App column

In [704]:
merged_df=pd.merge(app_df,reviews_df,on='App',how='inner')

In [ ]:
merged_df.head()

> ## 🔄 4. Data Transformation

* Transformation on Size column

In [706]:
def convert_size(size):
    if 'M' in size:
        return float(size.replace('M',''))
    elif 'k' in size:
        return float(size.replace('k',''))/1024
    else:
        return np.nan
app_df['Size']=app_df['Size'].apply(convert_size)

In [ ]:
app_df.head()

In [708]:
#Lograrithmic
app_df['Log_Installs']=np.log(app_df['Installs'])

* conver float 'Review' column in to int

In [616]:
app_df['Reviews']=app_df['Reviews'].astype(int)

In [ ]:
app_df.dtypes

In [711]:
app_df['Log_Reviews']=np.log(app_df['Reviews'])

In [ ]:
app_df.dtypes

In [713]:
def rating_group(rating):
    if rating >= 4:
        return 'Top rated app'
    elif rating >=3:
        return 'Above average'
    elif rating >=2:
        return 'Average'
    else:
        return 'Below Average'
app_df['Rating_Group']=app_df['Rating'].apply(rating_group)

In [714]:
#Revenue column
app_df['Revenue']=app_df['Price']*app_df['Installs']

## 🧮 5. Sentiment Analysis NLP

In [622]:
sia = SentimentIntensityAnalyzer() # a pre trained mode on Sentiments

- Polarity Scores in Sentiment Intensity Analyzer
    * Positive, 
    * Negative, 
    * Neutralm
    * Compound: -1 - Very negative ; +1 - Very positive

In [ ]:
# example
review = "This app is amazing! I love the new features."
sentiment_score= sia.polarity_scores(review)
print(sentiment_score)

In [ ]:
review = "This app is very bad! I hate the new features."
sentiment_score= sia.polarity_scores(review)
print(sentiment_score)

In [ ]:
review = "This app is okay."
sentiment_score= sia.polarity_scores(review)
print(sentiment_score)

---

In [719]:
reviews_df['Sentiment_Score']=reviews_df['Translated_Review'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

In [ ]:
reviews_df.head()

In [ ]:
app_df['Last Updated']=pd.to_datetime(app_df['Last Updated'],errors='coerce')
app_df['Year']=app_df['Last Updated'].dt.year
app_df.head()

> ## 📊 6. Plotly

In [722]:
html_files_path="./"
if not os.path.exists(html_files_path):
    os.makedirs(html_files_path)

In [723]:
plot_containers=""

In [724]:
# Save each Plotly figure to an HTML file
def save_plot_as_html(fig, filename, insight):
    global plot_containers
    filepath = os.path.join(html_files_path, filename)
    html_content = pio.to_html(fig, full_html=False, include_plotlyjs='inline')
    # Append the plot and its insight to plot_containers
    plot_containers += f"""
    <div class="plot-container" id="{filename}" onclick="openPlot('{filename}')">
        <div class="plot">{html_content}</div>
        <div class="insights">{insight}</div>
    </div>
    """
    fig.write_html(filepath, full_html=False, include_plotlyjs='inline')

In [725]:
plot_width=400
plot_height=300
plot_bg_color='black'
text_color='white'
title_font={'size':16}
axis_font={'size':12}

> ### Visualize Top 10 APP Category on Play Store

In [726]:
#Figure 1
category_counts=app_df['Category'].value_counts().nlargest(10)
fig1=px.bar(
    x=category_counts.index,
    y=category_counts.values,
    labels={'x':'Category','y':'Count'},
    title='Top Categories on Play Store',
    color=category_counts.index,
    color_discrete_sequence=px.colors.sequential.Plasma,
    width=400,
    height=300
)
fig1.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)

save_plot_as_html(fig1,"Category Graph 1.html","The top categories on the Play Store are dominated by tools, entertainment, and productivity apps")
            

> ### Distribution of Free v/ Paid Apps

In [727]:
#Figure 2
type_counts=app_df['Type'].value_counts()
fig2=px.pie(
    values=type_counts.values,
    names=type_counts.index,
    title='App Type Distribution',
    color_discrete_sequence=px.colors.sequential.RdBu,
    width=400,
    height=300
)
fig2.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig2,"Type Graph 2.html","Most apps on the Playstore are free, indicating a strategy to attract users first and monetize through ads or in app purchases")

> ### How the App Rating being Distribution

In [728]:
#Figure 3
fig3=px.histogram(
    app_df,
    x='Rating',
    nbins=20,
    title='Rating Distribution',
    color_discrete_sequence=['#636EFA'],
    width=400,
    height=300
)
fig3.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig3,"Rating Graph 3.html","Ratings are skewed towards higher values, suggesting that most apps are rated favorably by users")

> ### Sentiment Score

In [729]:
#Figure 4
sentiment_counts=reviews_df['Sentiment_Score'].value_counts()
fig4=px.bar(
    x=sentiment_counts.index,
    y=sentiment_counts.values,
    labels={'x':'Sentiment Score','y':'Count'},
    title='Sentiment Distribution',
    color=sentiment_counts.index,
    color_discrete_sequence=px.colors.sequential.RdPu,
    width=400,
    height=300
)
fig4.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig4,"Sentiment Graph 4.html","Sentiments in reviews show a mix of positive and negative feedback, with a slight lean towards positive sentiments")

> ### Which Category App Have Highest Installs

In [730]:
#Figure 5
installs_by_category=app_df.groupby('Category')['Installs'].sum().nlargest(10)
fig5=px.bar(
    x=installs_by_category.index,
    y=installs_by_category.values,
    orientation='h',
    labels={'x':'Installs','y':'Category'},
    title='Installs by Category',
    color=installs_by_category.index,
    color_discrete_sequence=px.colors.sequential.Blues,
    width=400,
    height=300
)
fig5.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig5,"Installs Graph 5.html","The categories with the most installs are social and communication apps, reflecting their broad appeal and daily usage")

> ### Number of Updates Over the Years

In [731]:
# Figure 6
updates_per_year = app_df['Last Updated'].dt.year.value_counts().sort_index()
fig6 = px.line(
    x=updates_per_year.index,
    y=updates_per_year.values,
    labels={'x': 'Year', 'y': 'Number of Updates'},
    title='Number of Updates Over the Years',
    color_discrete_sequence=['#AB63FA'],
    width=plot_width,
    height=plot_height
)
fig6.update_layout(
    plot_bgcolor=plot_bg_color,
    paper_bgcolor=plot_bg_color,
    font_color=text_color,
    title_font=title_font,
    xaxis=dict(title_font=axis_font),
    yaxis=dict(title_font=axis_font),
    margin=dict(l=10, r=10, t=30, b=10)
)
save_plot_as_html(fig6, "Updates Graph 6.html", "Updates have been increasing over the years, showing that developers are actively maintaining and improving their apps.")

> ### Revenue by Category

In [732]:
#Figure 7
revenue_by_category=app_df.groupby('Category')['Revenue'].sum().nlargest(10)
fig7=px.bar(
    x=installs_by_category.index,
    y=installs_by_category.values,
    labels={'x':'Category','y':'Revenue'},
    title='Revenue by Category',
    color=installs_by_category.index,
    color_discrete_sequence=px.colors.sequential.Greens,
    width=400,
    height=300
)
fig7.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig7,"Revenue Graph 7.html","Categories such as Business and Productivity lead in revenue generation, indicating their monetization potential")

> ### Top Genres

In [733]:
#Figure 8
genre_counts=app_df['Genres'].str.split(';',expand=True).stack().value_counts().nlargest(10)
fig8=px.bar(
    x=genre_counts.index,
    y=genre_counts.values,
    labels={'x':'Genre','y':'Count'},
    title='Top Genres',
    color=installs_by_category.index,
    color_discrete_sequence=px.colors.sequential.OrRd,
    width=400,
    height=300
)
fig8.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig8,"Genre Graph 8.html","Action and Casual genres are the most common, reflecting users' preference for engaging and easy-to-play games")

> ### Impact of Last Update on Rating

In [734]:
#Figure 9
fig9=px.scatter(
    app_df,
    x='Last Updated',
    y='Rating',
    color='Type',
    title='Impact of Last Update on Rating',
    color_discrete_sequence=px.colors.qualitative.Vivid,
    width=400,
    height=300
)
fig9.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig9,"Update Graph 9.html","The Scatter Plot shows a weak correlation between the last update and ratings, suggesting that more frequent updates dont always result in better ratings.")

> ### Rating for Paid vs Free Apps

In [735]:
#Figure 10
fig10=px.box(
    app_df,
    x='Type',
    y='Rating',
    color='Type',
    title='Rating for Paid vs Free Apps',
    color_discrete_sequence=px.colors.qualitative.Pastel,
    width=400,
    height=300
)
fig10.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig10,"Paid Free Graph 10.html","Paid apps generally have higher ratings compared to free apps, suggesting that users expect higher quality from apps they pay for")

> # Internship - Task 1

<code>Task 1: Create a scatter plot to visualize the relationship between revenue and the number of installs for paid apps only. Add a trendline to show the correlation and color-code the points based on app categories.</code>

In [ ]:
app_df.head()

In [ ]:
reviews_df.head()

* Filter only Paid apps

In [738]:
paid_apps = app_df[app_df['Type'] == 'Paid'].copy()

In [ ]:
paid_apps

In [740]:
paid_apps = paid_apps.dropna(subset=['Revenue', 'Installs', 'Category'])
paid_apps = paid_apps[(paid_apps['Revenue'] > 0) & (paid_apps['Installs'] > 0)]


In [741]:
top_categories = paid_apps['Category'].value_counts().nlargest(10).index
paid_apps = paid_apps[paid_apps['Category'].isin(top_categories)]


In [742]:
# Ensure 'Installs' and 'Revenue' are numeric
paid_apps['Installs'] = pd.to_numeric(paid_apps['Installs'], errors='coerce')
paid_apps['Revenue'] = pd.to_numeric(paid_apps['Revenue'], errors='coerce')
paid_apps = paid_apps.dropna(subset=['Installs', 'Revenue', 'Category'])

In [743]:
# SCATTER PLOT FOR PAID APPS
fig11 = px.scatter(
    paid_apps,
    x='Installs',
    y='Revenue',
    color='Category',
    trendline='ols',
    title='Revenue vs Installs for Paid Apps',
    labels={'Installs': 'Number of Installs', 'Revenue': 'App Revenue'},
    color_discrete_sequence=px.colors.qualitative.Set2,
    width=400,
    height=300
)

fig11.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font=title_font,
    xaxis=dict(title_font=axis_font),
    yaxis=dict(title_font=axis_font),
    margin=dict(l=10, r=10, t=30, b=10)
)

save_plot_as_html(
    fig11,
    "Scatter Graph 11.html",
    "Higher installs tend to correlate with increased revenue among paid apps, though certain categories outperform others regardless of install base."
)


---

> # Internship - Task 2

<code>Task 2. Generate a word cloud for the most frequent keywords found in 5-star reviews, but exclude common stopwords and app names. Additionally, filter the reviews to include only those from apps in the "Health & Fitness" category.</code>

### Step 0: Import Required Libraries

In [744]:
import pandas as pd
import string
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS


### Step 1: Basic Exploration

* View first 2 apps with a 5-star rating

In [ ]:
app_df[app_df['Rating'] == 5].head(2)

* View all unique app categories

In [ ]:
app_df['Category'].unique()


* View first few 5-star apps from HEALTH_AND_FITNESS category

In [ ]:
app_df[(app_df['Rating'] == 5) & (app_df['Category'] == 'HEALTH_AND_FITNESS')].head(2)


* Count of HEALTH_AND_FITNESS apps with 5-star rating

In [ ]:
count = app_df[(app_df['Rating'] == 5) & (app_df['Category'] == 'HEALTH_AND_FITNESS')].shape[0]
print(f"HEALTH_AND_FITNESS apps with Rating 5: {count}")


###  Step 2: Load and Preview Reviews Dataset

In [ ]:
reviews_df.head(2)

###  Step 3: Merge app data and review data

In [750]:
merge_df = pd.merge(reviews_df, app_df, on='App', how='inner')

###  Step 4: Filter HEALTH_AND_FITNESS apps with Rating = 5

In [ ]:
# Filter the merged DataFrame
filtered_df = merge_df[
    (merge_df['Category'] == 'HEALTH_AND_FITNESS') &
    (merge_df['Rating'] == 5.0)
]
filtered_df.head()


In [752]:
# Strip and lower-case app names
app_df['App'] = app_df['App'].str.strip().str.lower()
reviews_df['App'] = reviews_df['App'].str.strip().str.lower()


In [ ]:
merge_df = pd.merge(reviews_df, app_df, on='App', how='inner')

filtered_df = merge_df[
    (merge_df['Category'] == 'HEALTH_AND_FITNESS') &
    (merge_df['Rating'] == 5.0)
]
print(filtered_df.shape[0])  # Should be more than 0 now if there were matching apps


<code>No 5-star reviews found for apps in the Health & Fitness category in the dataset.
Hence, word cloud could not be generated for this segment.</code>

---

> # Internship - Task 3

<code>Task 3: Visualize the sentiment distribution (positive, neutral, negative) of user reviews using a stacked bar chart, segmented by rating groups (e.g., 1-2 stars, 3-4 stars, 4-5 stars). Include only apps with more than 1,000 reviews and group by the top 5 categories.</code>

In [754]:
import pandas as pd
import plotly.express as px

### Step 1: Clean and standardize app names

In [755]:
app_df['App'] = app_df['App'].str.strip().str.lower()
reviews_df['App'] = reviews_df['App'].str.strip().str.lower()

### Step 2: Merge datasets

In [756]:
merged_df = pd.merge(reviews_df, app_df, on='App', how='inner')

### Step 3: Filter apps with >1000 reviews

In [757]:
review_counts = merged_df['App'].value_counts()
apps_with_1000_reviews = review_counts[review_counts > 1000].index
filtered_df = merged_df[merged_df['App'].isin(apps_with_1000_reviews)]


### Step 4: Define rating groups

In [ ]:
def rating_group(r):
    if pd.isnull(r):
        return None
    elif r <= 2:
        return '1-2 Stars'
    elif r <= 4:
        return '3-4 Stars'
    else:
        return '4-5 Stars'

filtered_df['Rating Group'] = filtered_df['Rating'].apply(rating_group)


### Step 5: Filter to Top 5 Categories

In [759]:
top_5_categories = app_df['Category'].value_counts().head(5).index
filtered_df = filtered_df[filtered_df['Category'].isin(top_5_categories)]

### Step 6: Group by Category, Rating Group, and Sentiment


In [760]:
sentiment_counts = filtered_df.groupby(['Category', 'Rating Group', 'Sentiment_Score']).size().reset_index(name='Count')

### Step 7: Plotly stacked bar chart


In [761]:
# STACKED BAR SENTIMENT DISTRIBUTION
fig12 = px.bar(
    sentiment_counts,
    x='Category',
    y='Count',
    color='Sentiment_Score',
    barmode='stack',
    facet_col='Rating Group',
    color_discrete_sequence=px.colors.sequential.RdPu,
    labels={'Count': 'Review Count', 'Category': 'App Category'},
    title='Sentiment Distribution by Rating Group for Top 5 App Categories (Apps with >1000 Reviews)',
    width=400,
    height=300
)

fig12.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font=title_font,
    xaxis=dict(title_font=axis_font),
    yaxis=dict(title_font=axis_font),
    margin=dict(l=10, r=10, t=30, b=10)
)

fig12.update_xaxes(tickangle=45)

save_plot_as_html(
    fig12,
    "Sentiment Graph 12.html",
    "This stacked bar chart reveals how sentiment varies by rating groups across top app categories with more than 1000 reviews."
)


---

> # Internship - Task 4

<code> Task 4. Use a grouped bar chart to compare the average rating and total review count for the top 10 app categories by number of installs. Filter out any categories where the average rating is below 4.0 and size below 10 M and last update should be Jan month . this graph should work only between 3PM IST to 5 PM IST apart from that time we should not show this graph in dashboard itself.</code>

In [762]:
import pandas as pd
import plotly.express as px
from datetime import datetime
import pytz

Step 1: Convert relevant columns

In [763]:
# --- Step 1: Convert relevant columns ---
app_df['Installs'] = app_df['Installs'].replace('[+,]', '', regex=True).astype(int)
app_df['Size'] = app_df['Size'].replace('M', '', regex=False).replace('k', '', regex=False).replace('Varies with device', None)
app_df['Size'] = pd.to_numeric(app_df['Size'], errors='coerce')
app_df['Last Updated'] = pd.to_datetime(app_df['Last Updated'], errors='coerce')


In [764]:
# --- Step 2: Filter conditions ---
filtered_apps = app_df[
    (app_df['Rating'] >= 4.0) &
    (app_df['Size'] >= 10) &
    (app_df['Last Updated'].dt.month == 1)
]

In [ ]:
filtered_apps.head(2)

In [766]:
# --- Step 3: Group by Category and aggregate ---
category_grouped = filtered_apps.groupby('Category').agg({
    'Installs': 'sum',
    'Rating': 'mean',
    'Reviews': 'sum'
}).reset_index()


In [767]:
# --- Step 4: Top 10 categories by installs ---
top10 = category_grouped.sort_values(by='Installs', ascending=False).head(10)

In [768]:
# --- Step 5: Melt for grouped bar plot ---
melted = top10.melt(id_vars='Category', value_vars=['Rating', 'Reviews'], var_name='Metric', value_name='Value')


In [769]:
# --- Step 6: Time Check for 3–5 PM IST ---
ist = pytz.timezone('Asia/Kolkata')
now = datetime.now(ist)
show_plot = now.hour >= 15 and now.hour < 17


In [ ]:
# --- Step 7: Plot if within time ---
if show_plot:
    fig13 = px.bar(
        melted,
        x='Category',
        y='Value',
        color='Metric',
        barmode='group',
        title='Avg Rating & Total Reviews for Top 10 Categories (Installs-wise)',
        color_discrete_sequence=px.colors.qualitative.Vivid,
        labels={'Value': 'Metric Value'}
    )

    fig13.update_layout(
        plot_bgcolor='black',
        paper_bgcolor='black',
        font_color='white',
        title_font={'size': 18},
        xaxis=dict(title_font={'size': 14}),
        yaxis=dict(title_font={'size': 14}),
        margin=dict(l=30, r=30, t=50, b=30)
    )

    fig13.update_xaxes(tickangle=45)

    # Save the graph only if shown
    save_plot_as_html(
        fig13,
        "Grouped_Bar_Top10_Categories_3to5PM.html",
        "Grouped bar chart showing avg rating & review count for top 10 categories, only visible from 3–5 PM IST."
    )
else:
    print("Current IST time is outside 3 PM - 5 PM. Skipping graph.")

---

> # Internship - Task 5

<code> Task 5: Create an interactive Choropleth map using Plotly to visualize global installs by Category. Apply filters to show data for only the top 5 app categories and highlight category where the number of installs exceeds 1 million. The app category should not start with the characters “A,” “C,” “G,” or “S.” This graph should work only between 6 PM IST and 8 PM IST; apart from that time, we should not show it in the dashboard itself.</code>

In [771]:
# --- Step 1: Time Gate (6 PM – 8 PM IST) ---
ist = pytz.timezone('Asia/Kolkata')
now = datetime.now(ist)
show_choropleth = 18 <= now.hour < 20

In [ ]:
if show_choropleth:
    # --- Step 2: Preprocessing ---

    # Clean installs
    app_df['Installs'] = app_df['Installs'].replace('[+,]', '', regex=True).astype(int)

    # Remove categories starting with A, C, G, S
    app_df = app_df[~app_df['Category'].str.startswith(tuple('ACGS'))]

    # Filter: only installs > 1M
    app_df = app_df[app_df['Installs'] > 1_000_000]

    # For demonstration, assume a Country column exists
    # If not, simulate one for visualization
    if 'Country' not in app_df.columns:
        import numpy as np
        countries = ['IN', 'US', 'GB', 'CA', 'AU', 'BR', 'DE', 'FR', 'RU', 'JP']
        app_df['Country'] = np.random.choice(countries, size=len(app_df))

    # --- Step 3: Get Top 5 categories by total installs ---
    top_categories = (
        app_df.groupby('Category')['Installs']
        .sum()
        .sort_values(ascending=False)
        .head(5)
        .index
    )

    filtered = app_df[app_df['Category'].isin(top_categories)]

    # --- Step 4: Aggregate installs by Country + Category ---
    grouped = (
        filtered.groupby(['Country', 'Category'])['Installs']
        .sum()
        .reset_index()
    )

    # --- Step 5: Choropleth Map ---
    fig14 = px.choropleth(
        grouped,
        locations='Country',
        color='Installs',
        hover_name='Category',
        locationmode='ISO-3',
        color_continuous_scale=px.colors.sequential.YlOrRd,
        labels={'Installs': 'Install Count'},
        title='Global Installs by App Category (Top 5 Only, >1M installs)',
        height=600,
        width=1000
    )

    fig14.update_layout(
        geo=dict(
            showframe=False,
            showcoastlines=True,
            projection_type='equirectangular'
        ),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font_color='white',
        title_font={'size': 20},
        margin=dict(l=30, r=30, t=50, b=30)
    )

    # --- Step 6: Save ---
    save_plot_as_html(
        fig14,
        "Choropleth_Installs_By_Category.html",
        "Choropleth map of installs by country and category (Top 5 categories >1M installs, category names not starting with A/C/G/S). Shown only 6–8 PM IST."
    )

else:
    print("Current IST time is outside 6 - 8 PM. Choropleth map will not be generated.")

---

> # Internship - Task 6
<code> Task 6: Create a dual-axis chart comparing the average installs and revenue for free vs. paid apps within the top 3 app categories. Apply filters to exclude apps with fewer than 10,000 installs and revenue below $10,000 and android version should be more than 4.0 as well as size should be more than 15M and content rating should be Everyone and app name should not have more than 30 characters including space and special character .this graph should work only between 1 PM IST to 2 PM IST apart from that time we should not show this graph in dashboard itself.</code>

In [773]:
import plotly.graph_objects as go
from datetime import datetime
import pytz

# Set IST timezone
ist = pytz.timezone('Asia/Kolkata')
now = datetime.now(ist)
current_hour = now.hour

# Task 6 should only run between 1 PM and 2 PM IST
if 13 <= current_hour < 14:
    
    # Clean Type column to ensure consistent values
    app_df['Type'] = app_df['Type'].astype(str).str.strip().str.capitalize()

    # Filter as per task
    filtered_df = app_df[
    (app_df['Installs'] >= 10000) &
    (app_df['Revenue'] >= 10000) &
    (app_df['Android Ver'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)[0] > 4.0) &
    (app_df['Size'].astype(str).str.replace('M', '', regex=False).replace('Varies with device', np.nan).astype(float) > 15) &
    (app_df['Content Rating'] == 'Everyone') &
    (app_df['App'].astype(str).str.len() <= 30)
    ]


    # Get top 3 categories by count
    top_categories = (
        filtered_df['Category']
        .value_counts()
        .head(3)
        .index
    )

    filtered_df = filtered_df[filtered_df['Category'].isin(top_categories)]

    # Create pivot tables
    installs_pivot = filtered_df.pivot_table(index='Category', columns='Type', values='Installs', aggfunc='mean')
    revenue_pivot = filtered_df.pivot_table(index='Category', columns='Type', values='Revenue', aggfunc='mean')

    # Plot
    fig15 = go.Figure()

    # Bar for Installs
    for t in ['Free', 'Paid']:
        if t in installs_pivot.columns:
            fig15.add_trace(go.Bar(
                x=installs_pivot.index,
                y=installs_pivot[t],
                name=f'Avg Installs ({t})',
                marker=dict(color='lightblue' if t == 'Free' else 'lightgreen'),
                yaxis='y1'
            ))

    # Line for Revenue
    for t in ['Free', 'Paid']:
        if t in revenue_pivot.columns:
            fig15.add_trace(go.Scatter(
                x=revenue_pivot.index,
                y=revenue_pivot[t],
                name=f'Avg Revenue ({t})',
                mode='lines+markers',
                marker=dict(symbol='circle'),
                yaxis='y2'
            ))

    # Layout
    fig15.update_layout(
        title='Avg Installs vs Revenue by App Type (Top 3 Categories)',
        xaxis=dict(title='App Category'),
        yaxis=dict(title='Average Installs', side='left'),
        yaxis2=dict(title='Average Revenue ($)', overlaying='y', side='right'),
        barmode='group',
        plot_bgcolor='black',
        paper_bgcolor='black',
        font_color='white',
        title_font=title_font,
        xaxis_title_font=axis_font,
        yaxis_title_font=axis_font,
        yaxis2_title_font=axis_font,
        legend=dict(bgcolor='black'),
        width=600,
        height=400,
        margin=dict(l=10, r=10, t=30, b=10)
    )

    # Save
    save_plot_as_html(
        fig15,
        "Dual Axis Graph 15.html",
        "Dual-axis chart comparing installs and revenue for Free vs Paid apps in top 3 categories (filtered by installs, revenue, android version, size, content rating, app name length)."
    )

else:
    print("Task 6: Skipped. Not in valid time window (1 PM – 2 PM IST).")


---

In [681]:
plot_containers_split=plot_containers.split('</div>')

In [682]:
if len(plot_containers_split) > 1:
    final_plot=plot_containers_split[-2]+'</div>'
else:
    final_plot=plot_containers

In [780]:
dashboard_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Google Play Store Review Analytics</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #333;
            color: #fff;
            margin: 0;
            padding: 0;
        }}
        .header {{
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 20px;
            background-color: #444;
            flex-wrap: wrap;
            text-align: center;
        }}
        .header img {{
            margin: 10px;
            height: 50px;
        }}
        .container {{
            display: flex;
            flex-wrap: wrap;
            justify-content: center;
            padding: 20px;
        }}
        .plot-container {{
            border: 2px solid #555;
            margin: 10px;
            padding: 10px;
            width: {plot_width}px;
            height: {plot_height}px;
            overflow: auto;
            position: relative;
            background-color: #222;
            border-radius: 8px;
        }}
        .insights {{
            display: none;
            position: absolute;
            right: 10px;
            top: 10px;
            background-color: rgba(0,0,0,0.7);
            padding: 5px;
            border-radius: 5px;
            color: #fff;
        }}
        .plot-container:hover .insights {{
            display: block;
        }}
    </style>
    <script>
        function openPlot(filename) {{
            window.open(filename, '_blank');
        }}
    </script>
</head>
<body>
    <div class="header">
        <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4a/Logo_2013_Google.png/800px-Logo_2013_Google.png" alt="Google Logo">
        <h1>Google Play Store Reviews Analytics</h1>
        <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/78/Google_Play_Store_badge_EN.svg/1024px-Google_Play_Store_badge_EN.svg.png" alt="Google Play Store Logo">
    </div>
    <div class="container">
        {plots}
    </div>
</body>
</html>
"""


In [781]:
final_html=dashboard_html.format(plots=plot_containers,plot_width=plot_width,plot_height=plot_height)

In [782]:
dashboard_path=os.path.join(html_files_path,"web page.html")

In [783]:
with open(dashboard_path, "w", encoding="utf-8") as f:
    f.write(final_html)